In [1]:
%matplotlib inline
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas as pd

from game import Game
from player import Player

In [2]:
from your_name import YourName

In [3]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]


In [4]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(11, 0), (12, 0), (10, 1), (12, 3), (13, 3), (11, 2), (10, 0), (13, 2)]
[(11, 1), (9, 0), (12, 1), (11, 3), (13, 0), (14, 3), (12, 2), (14, 2)]


In [5]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

Analyze few moves...

In [6]:
### Perform a full game 100 times
STAT_NAMES = ["Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"]
ANALYZE_STATS = [0, 1, 2, 3, 5, 6, 7, 8]

def printResults(results):
    print("Wins:")
    print(results[0])
    print("Draws:")
    print(results[1])
    print("Moves:")
    print(stats_moves[2])
    print("Cards:")
    print(results[3])
    print("Pile size:")
    print(results[4])
    print("Checks:")
    print(results[5])
    print("Draw decisions:")
    print(results[6])
    print("Cheats:")
    print(results[7])
    print("Errors:")
    print(results[8])
    print("Total errors:")
    print(results[9])

def comparePlayers(player1_class, player2_class):
    stats_wins = [0, 0]
    stats_draws = [0, 0]
    stats_moves = [0, 0]
    stats_cards = [0, 0]
    stats_pile_size = 0
    stats_checks = [0, 0]
    stats_draw_decisions = [0, 0]
    stats_cheats = [0, 0]
    stats_errors = [0, 0]
    
    repeats = 1000
    errors = 0
    draws = 0

    for t in range(repeats):
        player1 = player1_class("")
        player2 = player2_class("")
        game = Game([player1, player2], log = False)
    
        error = False
        draw = False
        
        while True:
            valid, player = game.takeTurn(log = False)
            if game.moves[0] > 100 or game.moves[1] > 100:
                draws += 1
                stats_draws[0] += 1
                stats_draws[1] += 1
                if (game.player_cards[0] < game.player_cards[1]):
                    stats_wins[0] += 1
                if (game.player_cards[0] > game.player_cards[1]):
                    stats_wins[1] += 1
                    
                    
                draw=True
                #print("DRAW")
                break
            if not valid:
                error = True
                stats_errors[player] += 1
                errors += 1
                break
            if game.isFinished(log = False):
                stats_wins[player] += 1
                break
            
        stats_pile_size += len(game.pile)
        if error: continue
        #if draw: continue
       
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

    div = repeats - errors
    if div > 0:
            
        stats_pile_size /= div          
        for j in range(2):
            stats_moves[j] /= div
            stats_cheats[j] /= div
            stats_checks[j] /= div
            stats_draw_decisions[j] /= div
            stats_cards[j] /= div
            
    return [stats_wins, stats_draws, stats_moves, stats_cards, stats_pile_size, stats_checks, 
            stats_draw_decisions, stats_cheats, stats_errors, errors, draws]  


# COMPARE

In [7]:
strategy = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["YourName", "YourName", YourName],
           ]

In [8]:
#%pdb on
full_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]

for A in range(len(strategy)):
    print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        full_results[A][B] = results
        


==== 0 RandomPlayer
1 YourName
==== 1 YourName


In [9]:
full_results

[[None,
  [[1000, 0],
   [0, 0],
   [8.0, 7.504],
   [0.0, 15.0],
   1.0,
   [0.0, 0.0],
   [0.0, 7.504],
   [2.64, 0.0],
   [0, 0],
   0,
   0]],
 [None, None]]

Simple stats

In [10]:
def printMatrix(full_results, stat):
    print(STAT_NAMES[stat])
    S = " "
    for s in strategy: S += (str(s[1]) + " " )
    print(S)
    for A in range(len(strategy)):
        print(A)
        S = str(strategy[A][1]) + " "
        for B in range(len(strategy)):
            if A == B: S += "- "
            elif A < B:
                S += str(full_results[A][B][stat][0]) + " "
            else:
                S += str(full_results[B][A][stat][1]) + " "
        print(S)
    
for a in ANALYZE_STATS:
    printMatrix(full_results, a)



Wins
 RANDOM YourName 
0
RANDOM - 1000 
1
YourName 0 - 
Draws
 RANDOM YourName 
0
RANDOM - 0 
1
YourName 0 - 
Moves
 RANDOM YourName 
0
RANDOM - 8.0 
1
YourName 7.504 - 
Cards
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 15.0 - 
Checks
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 0.0 - 
Draw Decisions
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 7.504 - 
Cheats
 RANDOM YourName 
0
RANDOM - 2.64 
1
YourName 0.0 - 
Errors
 RANDOM YourName 
0
RANDOM - 0 
1
YourName 0 - 


### __________________________________________My Code ______________________________________________

In [11]:
agent_strategies = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["YourName", "YourName", YourName],
           ["RandomPlayer2", "RANDOM", RandomPlayer]]

def compare_all(agent_strategies, verbose = 1):
    wins = {agent_strategies[i][0]: {agent_strategies[i][0]: None} for i in range(len(agent_strategies))}
    for i in range(len(agent_strategies)):
        for ii in range(i+1, len(agent_strategies)):
            player1 = agent_strategies[i]
            player2 = agent_strategies[ii]
            strategy = [player1, player2]
            for A in range(len(strategy)):
                for B in range(A+1,len(strategy)):
                    results = comparePlayers(strategy[A][2], strategy[B][2])
                    full_results[A][B] = results
                    wins[strategy[A][0]][strategy[B][0]] = full_results[0][1][0][1]
                    wins[strategy[B][0]][strategy[A][0]] = full_results[0][1][0][0]
                    if verbose:
                        if full_results[0][1][8][0] > 0 or full_results[0][1][8][1] > 0 or full_results[0][1][9] > 0:
                            print(f"Errors occured in game: {strategy[A][0]} vs {strategy[B][0]}")
    df = pd.DataFrame(wins)
    df['|| Total Wins ||'] = df.fillna(0).sum(axis=1)
    df = df.sort_values(by='|| Total Wins ||', ascending=False)

    dominated_players = []
    temp_df = df.drop(columns=["|| Total Wins ||"])
    for i, row in temp_df.iterrows():
        num_dominated = 0
        for column in temp_df.columns:
            if i != column and temp_df[column][i] > temp_df[i][column]:
                num_dominated += 1
        dominated_players.append(num_dominated)
    df['|| Dominated Players ||'] = dominated_players
    return df

compare_all(agent_strategies)

,RandomPlayer,YourName,RandomPlayer2,|| Total Wins ||,|| Dominated Players ||
RandomPlayer2,518.0,1000.0,NaN,1518.0,2
RandomPlayer,NaN,1000.0,482.0,1482.0,1
YourName,0.0,NaN,0.0,0.0,0


In [12]:
from Test_Players import *
from B import *
from adamK import *
from Adam_player import *
from ensamble import *

In [13]:
agent_strategies = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["Saint_Sceptic_25_Naive", "SAINT", Saint_Sceptic_25_Naive],
            ["Saint_Sceptic_50_Naive", "SAINT", Saint_Sceptic_50_Naive],
            ["Saint_Sceptic_75_Naive", "SAINT", Saint_Sceptic_75_Naive],
            ["Saint_Sceptic_Adaptable_Naive", "SAINT", Saint_Sceptic_Adaptable_Naive],
            ["Ensamble", "ENSAMBLE", ensamble_player],
            ["YourName", "YourName", YourName],
           ["MyRandom_Default", "CUSTOM_RANDOM", MyRandom_Default],
           ["MyRandom_All_Cards", "CUSTOM_RANDOM", MyRandom_All_Cards],
           ["MyRandom_Real_Cards", "CUSTOM_RANDOM", MyRandom_Real_Cards],
           ["Random_Never_Accuse", "CUSTOM_RANDOM", Random_Never_Accuse],
           ["Random_Always_Accuse", "CUSTOM_RANDOM", Random_Always_Accuse],
           ["Mimic", "CUSTOM_RANDOM", Mimic],
           ["Saint_Believer", "SAINT", Saint_Believer],
           ["Saint_Once_in_a_While", "SAINT", Saint_Once_in_a_While],
           ["Saint_Two_Face", "SAINT", Saint_Two_Face],
           ["Saint_Accountant", "SAINT", Saint_Accountant],
           ["Saint_Nervous", "SAINT", Saint_Nervous],
           ["Saint_Nervous_ver_2", "SAINT", Saint_Nervous_ver_2],
           ["Saint_Nervous_ver_3", "SAINT", Saint_Nervous_ver_3],
           ["Saint_Nervous_Memory", "SAINT", Saint_Nervous_Memory],
           ["Adam_player", "CUSTOM", Adam_player],
           ["AdamK", "CUSTOM", AdamK],
           ["AdamK2", "CUSTOM", AdamK2],
           ["Saint_Collector", "SAINT", Saint_Collector],
           ["Dabbler_Believer", "DABBLER", Dabbler_Believer],
           ["Dabbler_Once_in_a_While", "DABBLER", Dabbler_Once_in_a_While],
           ["Dabbler_Two_Face", "DABBLER", Dabbler_Two_Face],
           ["Dabbler_Accountant", "DABBLER", Dabbler_Accountant],
           ["Dabbler_Nervous", "DABBLER", Dabbler_Nervous],
           ["Dabbler_Collector", "DABBLER", Dabbler_Collector],
           ["Coin_Flipper_Believer", "COIN_FLIPPER", Coin_Flipper_Believer],
           ["Coin_Flipper_Once_in_a_While", "COIN_FLIPPER", Coin_Flipper_Once_in_a_While],
           ["Coin_Flipper_Two_Face", "COIN_FLIPPER", Coin_Flipper_Two_Face],
           ["Coin_Flipper_Accountant", "COIN_FLIPPER", Coin_Flipper_Accountant],
           ["Coin_Flipper_Nervous", "COIN_FLIPPER", Coin_Flipper_Nervous],
           ["Coin_Flipper_Collector", "COIN_FLIPPER", Coin_Flipper_Collector],
           ["Pathological_Believer", "PATHOLOGICAL", Pathological_Believer],
           ["Pathological_Once_in_a_While", "PATHOLOGICAL", Pathological_Once_in_a_While],
           ["Pathological_Two_Face", "PATHOLOGICAL", Pathological_Two_Face],
           ["Pathological_Accountant", "PATHOLOGICAL", Pathological_Accountant],
           ["Pathological_Nervous", "PUSHER", Pathological_Nervous],
           ["Pathological_Collector", "PUSHER", Pathological_Collector],
           ["Pusher_Believer", "PUSHER", Pusher_Believer],
           ["Pusher_Once_in_a_While", "PUSHER", Pusher_Once_in_a_While],
           ["Pusher_Two_Face", "PUSHER", Pusher_Two_Face],
           ["Pusher_Accountant", "PUSHER", Pusher_Accountant],
           ["Pusher_Nervous", "PUSHER", Pusher_Nervous],
           ["Pusher_Collector", "PUSHER", Pusher_Collector],
           ["B_1", "B_PLAYER", Blayer],
           ["B_2", "B_PLAYER", Blayer2],
           ["B_3", "B_PLAYER", Blayer3],
           ["Saint_Nervous_ver_B", "B_PLAYER", Saint_Nervous_ver_B],
           ["Saint_Nervous_ver_B2", "B_PLAYER", Saint_Nervous_ver_B2],
           ["Saint_Nervous_ver_B3", "B_PLAYER", Saint_Nervous_ver_B3],
           ["Saint_Nervous_ver_B4", "B_PLAYER", Saint_Nervous_ver_B4],
           ]

results = compare_all(agent_strategies)
results

,RandomPlayer,Saint_Sceptic_25_Naive,Saint_Sceptic_50_Naive,Saint_Sceptic_75_Naive,Saint_Sceptic_Adaptable_Naive,Ensamble,YourName,MyRandom_Default,MyRandom_All_Cards,MyRandom_Real_Cards,...,Pusher_Collector,B_1,B_2,B_3,Saint_Nervous_ver_B,Saint_Nervous_ver_B2,Saint_Nervous_ver_B3,Saint_Nervous_ver_B4,|| Total Wins ||,|| Dominated Players ||
Saint_Nervous_ver_B,1000.0,1000.0,1000.0,1000.0,838.0,508.0,1000.0,1000.0,1000.0,1000.0,...,959.0,679.0,650.0,586.0,NaN,493.0,534.0,541.0,47415.0,52
Saint_Nervous_ver_B2,1000.0,1000.0,1000.0,1000.0,837.0,511.0,1000.0,1000.0,1000.0,1000.0,...,955.0,620.0,631.0,587.0,507.0,NaN,520.0,533.0,47359.0,55
Ensamble,1000.0,1000.0,1000.0,1000.0,826.0,NaN,1000.0,1000.0,1000.0,1000.0,...,955.0,622.0,657.0,553.0,492.0,489.0,538.0,544.0,47316.0,51
Saint_Nervous_ver_2,1000.0,1000.0,1000.0,1000.0,809.0,467.0,1000.0,1000.0,1000.0,1000.0,...,957.0,661.0,675.0,458.0,515.0,490.0,457.0,453.0,46924.0,49
Saint_Nervous_ver_3,1000.0,1000.0,1000.0,1000.0,820.0,500.0,1000.0,1000.0,1000.0,1000.0,...,957.0,633.0,650.0,493.0,506.0,489.0,456.0,483.0,46868.0,49
Saint_Nervous,1000.0,1000.0,1000.0,1000.0,819.0,506.0,1000.0,1000.0,1000.0,1000.0,...,936.0,661.0,668.0,360.0,485.0,485.0,376.0,378.0,45639.0,49
Saint_Nervous_ver_B3,927.0,924.0,916.0,927.0,969.0,462.0,1000.0,935.0,994.0,817.0,...,798.0,615.0,599.0,507.0,466.0,480.0,NaN,514.0,44719.0,52
Saint_Nervous_ver_B4,928.0,921.0,928.0,943.0,975.0,456.0,1000.0,905.0,994.0,754.0,...,795.0,616.0,610.0,497.0,459.0,467.0,486.0,NaN,44680.0,50
Saint_Nervous_Memory,1000.0,918.0,933.0,921.0,592.0,278.0,1000.0,1000.0,1000.0,1000.0,...,951.0,371.0,339.0,482.0,265.0,254.0,441.0,457.0,42881.0,42
B_3,877.0,932.0,933.0,921.0,910.0,447.0,1000.0,892.0,989.0,761.0,...,629.0,593.0,602.0,NaN,414.0,413.0,493.0,503.0,41892.0,50


In [14]:
results["AdamK"]

Saint_Nervous_ver_B              642.0
Saint_Nervous_ver_B2             634.0
Ensamble                         659.0
Saint_Nervous_ver_2              634.0
Saint_Nervous_ver_3              632.0
Saint_Nervous                    635.0
Saint_Nervous_ver_B3             522.0
Saint_Nervous_ver_B4             504.0
Saint_Nervous_Memory             480.0
B_3                              516.0
AdamK                              NaN
AdamK2                           442.0
Dabbler_Nervous                  288.0
Saint_Accountant                 125.0
Saint_Two_Face                    68.0
Saint_Sceptic_Adaptable_Naive    222.0
Dabbler_Accountant               103.0
Dabbler_Two_Face                  53.0
Adam_player                       67.0
Pusher_Accountant                355.0
B_1                              414.0
Saint_Once_in_a_While            307.0
B_2                              431.0
Pusher_Two_Face                  445.0
Saint_Sceptic_25_Naive            37.0
Coin_Flipper_Accountant  

In [15]:
results["AdamK2"]

Saint_Nervous_ver_B              698.0
Saint_Nervous_ver_B2             666.0
Ensamble                         660.0
Saint_Nervous_ver_2              644.0
Saint_Nervous_ver_3              615.0
Saint_Nervous                    640.0
Saint_Nervous_ver_B3             576.0
Saint_Nervous_ver_B4             582.0
Saint_Nervous_Memory             478.0
B_3                              565.0
AdamK                            558.0
AdamK2                             NaN
Dabbler_Nervous                  317.0
Saint_Accountant                 178.0
Saint_Two_Face                   151.0
Saint_Sceptic_Adaptable_Naive    248.0
Dabbler_Accountant               103.0
Dabbler_Two_Face                  79.0
Adam_player                       65.0
Pusher_Accountant                400.0
B_1                              525.0
Saint_Once_in_a_While            296.0
B_2                              522.0
Pusher_Two_Face                  517.0
Saint_Sceptic_25_Naive            38.0
Coin_Flipper_Accountant  

In [16]:
results["Saint_Nervous_ver_B2"]

Saint_Nervous_ver_B              493.0
Saint_Nervous_ver_B2               NaN
Ensamble                         489.0
Saint_Nervous_ver_2              490.0
Saint_Nervous_ver_3              489.0
Saint_Nervous                    485.0
Saint_Nervous_ver_B3             480.0
Saint_Nervous_ver_B4             467.0
Saint_Nervous_Memory             254.0
B_3                              413.0
AdamK                            366.0
AdamK2                           334.0
Dabbler_Nervous                  161.0
Saint_Accountant                  56.0
Saint_Two_Face                     4.0
Saint_Sceptic_Adaptable_Naive    163.0
Dabbler_Accountant                17.0
Dabbler_Two_Face                   0.0
Adam_player                        0.0
Pusher_Accountant                  4.0
B_1                              380.0
Saint_Once_in_a_While            225.0
B_2                              369.0
Pusher_Two_Face                    0.0
Saint_Sceptic_25_Naive             0.0
Coin_Flipper_Accountant  

In [17]:
results["Saint_Nervous_ver_B"]

Saint_Nervous_ver_B                NaN
Saint_Nervous_ver_B2             507.0
Ensamble                         492.0
Saint_Nervous_ver_2              515.0
Saint_Nervous_ver_3              506.0
Saint_Nervous                    485.0
Saint_Nervous_ver_B3             466.0
Saint_Nervous_ver_B4             459.0
Saint_Nervous_Memory             265.0
B_3                              414.0
AdamK                            358.0
AdamK2                           302.0
Dabbler_Nervous                  153.0
Saint_Accountant                  53.0
Saint_Two_Face                     3.0
Saint_Sceptic_Adaptable_Naive    162.0
Dabbler_Accountant                14.0
Dabbler_Two_Face                   0.0
Adam_player                        2.0
Pusher_Accountant                  3.0
B_1                              321.0
Saint_Once_in_a_While            237.0
B_2                              350.0
Pusher_Two_Face                    0.0
Saint_Sceptic_25_Naive             0.0
Coin_Flipper_Accountant  

In [18]:
results["Saint_Nervous_ver_3"]

Saint_Nervous_ver_B              494.0
Saint_Nervous_ver_B2             511.0
Ensamble                         500.0
Saint_Nervous_ver_2              520.0
Saint_Nervous_ver_3                NaN
Saint_Nervous                    482.0
Saint_Nervous_ver_B3             544.0
Saint_Nervous_ver_B4             517.0
Saint_Nervous_Memory             249.0
B_3                              507.0
AdamK                            368.0
AdamK2                           385.0
Dabbler_Nervous                  164.0
Saint_Accountant                  69.0
Saint_Two_Face                     4.0
Saint_Sceptic_Adaptable_Naive    180.0
Dabbler_Accountant                22.0
Dabbler_Two_Face                   0.0
Adam_player                        0.0
Pusher_Accountant                  3.0
B_1                              367.0
Saint_Once_in_a_While            215.0
B_2                              350.0
Pusher_Two_Face                    1.0
Saint_Sceptic_25_Naive             0.0
Coin_Flipper_Accountant  

In [19]:
results["Saint_Nervous_ver_2"]

Saint_Nervous_ver_B              485.0
Saint_Nervous_ver_B2             510.0
Ensamble                         533.0
Saint_Nervous_ver_2                NaN
Saint_Nervous_ver_3              480.0
Saint_Nervous                    504.0
Saint_Nervous_ver_B3             543.0
Saint_Nervous_ver_B4             547.0
Saint_Nervous_Memory             268.0
B_3                              542.0
AdamK                            366.0
AdamK2                           356.0
Dabbler_Nervous                  170.0
Saint_Accountant                  58.0
Saint_Two_Face                     2.0
Saint_Sceptic_Adaptable_Naive    191.0
Dabbler_Accountant                17.0
Dabbler_Two_Face                   0.0
Adam_player                        1.0
Pusher_Accountant                  4.0
B_1                              339.0
Saint_Once_in_a_While            232.0
B_2                              325.0
Pusher_Two_Face                    0.0
Saint_Sceptic_25_Naive             0.0
Coin_Flipper_Accountant  

In [20]:
results["Saint_Nervous_ver_B4"]

Saint_Nervous_ver_B              541.0
Saint_Nervous_ver_B2             533.0
Ensamble                         544.0
Saint_Nervous_ver_2              453.0
Saint_Nervous_ver_3              483.0
Saint_Nervous                    378.0
Saint_Nervous_ver_B3             514.0
Saint_Nervous_ver_B4               NaN
Saint_Nervous_Memory             457.0
B_3                              503.0
AdamK                            496.0
AdamK2                           418.0
Dabbler_Nervous                  205.0
Saint_Accountant                  30.0
Saint_Two_Face                    55.0
Saint_Sceptic_Adaptable_Naive     25.0
Dabbler_Accountant                15.0
Dabbler_Two_Face                  47.0
Adam_player                       53.0
Pusher_Accountant                245.0
B_1                              384.0
Saint_Once_in_a_While            169.0
B_2                              390.0
Pusher_Two_Face                  380.0
Saint_Sceptic_25_Naive            79.0
Coin_Flipper_Accountant  

In [21]:
results["Saint_Nervous"]

Saint_Nervous_ver_B              515.0
Saint_Nervous_ver_B2             515.0
Ensamble                         494.0
Saint_Nervous_ver_2              496.0
Saint_Nervous_ver_3              518.0
Saint_Nervous                      NaN
Saint_Nervous_ver_B3             624.0
Saint_Nervous_ver_B4             622.0
Saint_Nervous_Memory             229.0
B_3                              640.0
AdamK                            365.0
AdamK2                           360.0
Dabbler_Nervous                  347.0
Saint_Accountant                  52.0
Saint_Two_Face                     5.0
Saint_Sceptic_Adaptable_Naive    181.0
Dabbler_Accountant                44.0
Dabbler_Two_Face                   0.0
Adam_player                        1.0
Pusher_Accountant                  6.0
B_1                              339.0
Saint_Once_in_a_While            229.0
B_2                              332.0
Pusher_Two_Face                    0.0
Saint_Sceptic_25_Naive             0.0
Coin_Flipper_Accountant  